In [21]:
import requests
import json

In [ ]:
# Robust logging configuration
import logging
import sys

# Remove all handlers associated with the root logger object
for handler in logging.root.handlers[:]:
    logging.root.removeHandler(handler)

LOG_FORMAT = '%(asctime)s - %(levelname)s - %(name)s - %(message)s'
LOG_LEVEL = logging.DEBUG

# Create handlers
console_handler = logging.StreamHandler(sys.stdout)
console_handler.setLevel(LOG_LEVEL)
console_handler.setFormatter(logging.Formatter(LOG_FORMAT))


# Get root logger and set level
logger = logging.getLogger()
logger.setLevel(LOG_LEVEL)
logging.getLogger("urllib3.connectionpool").setLevel(logging.WARNING)

# Add handlers if not already present
if not any(isinstance(h, logging.StreamHandler) for h in logger.handlers):
    logger.addHandler(console_handler)

# Example usage:
logger.info('Logging is configured. Console and file output enabled.')

2025-07-31 19:28:45,134 - INFO - root - Logging is configured. Console and file output enabled.


In [23]:
UPSTREAM_REPO = "actions/runner-images"

In [24]:
# releases_page = requests.get(
#             f"https://api.github.com/repos/{UPSTREAM_REPO}/releases",
#             params={'per_page': 1, 'page': 1}
#         )

# releases_page.json()


In [25]:
from typing import Optional, Iterable
def fetch_last_software_report(templates: Optional[Iterable[str]] = None, repo: str = UPSTREAM_REPO, max_releases: int = 10) -> dict[str, dict]:
    """
    Fetches latest Runner Images software reports (internal.{os}.json) from the upstream repository.

    Args:
        templates (Iterable[str], optional): Iterable of templates to get software report internal json for. If None or empty, fetches unique internal.{os}.json for last max_releases releases.
        repo (str): The GitHub repository to fetch the reports from.
        max_releases (int): Number of releases to check if templates is empty.
    Returns:
        dict[str, dict]: A dictionary where keys are template names and values are the contents of the software report for that template.
    Example:
        >>> fetch_last_software_report(['win25', 'ubuntu24'])
        {
            'win25': {"NodeType": "HeaderNode", "Title": "Windows Server 2025", ...},
            'ubuntu24': {"NodeType": "HeaderNode", "Title": "Ubuntu 24.04", ...}
        }
        >>> fetch_last_software_report()
        { ...unique reports for last max_releases releases... }
    """

    reports = {}
    page = 1
    releases_checked = 0
    if templates:
        templates = set(templates)
    else:
        templates = set()

    while (templates or not templates) and releases_checked < max_releases:
        try:
            releases_page = requests.get(
                f"https://api.github.com/repos/{repo}/releases",
                params={'per_page': 100, 'page': page}
            )
            releases_page.raise_for_status()
        except requests.exceptions.RequestException as e:
            logger.error(f"Error fetching releases: {e}")
            break

        releases = releases_page.json()
        if not releases:
            break  # No more pages
        for release in releases:
            if releases_checked >= max_releases:
                break
            tag_name = release.get('tag_name', '')
            template_key = tag_name.split('/')[0]
            if template_key in reports:
                continue  # Already fetched, skip further processing
            found_asset = False
            for asset in release.get('assets', []):
                if asset['name'].startswith('internal.') and asset['name'].endswith('.json'):
                    if not templates or template_key in templates:
                        try:
                            response = requests.get(asset['browser_download_url'])
                            response.raise_for_status()
                            reports[template_key] = response.json()
                            logger.info(f"Fetched report for {template_key}")
                            found_asset = True
                        except requests.exceptions.RequestException as e:
                            logger.error(f"Error fetching asset for {template_key}: {e}")
                    break
            if not found_asset and (not templates or template_key in templates):
                logger.warning(f"No internal asset found for {template_key} in release {tag_name}")
            if templates and template_key in templates:
                templates.remove(template_key)
            # Only increment releases_checked once per release
            releases_checked += 1
        page += 1

    return reports


In [26]:
reports = fetch_last_software_report()

2025-07-31 19:28:46,369 - INFO - root - Fetched report for ubuntu24
2025-07-31 19:28:46,810 - INFO - root - Fetched report for ubuntu22
2025-07-31 19:28:47,255 - INFO - root - Fetched report for macos-14-arm64
2025-07-31 19:28:47,701 - INFO - root - Fetched report for win25
2025-07-31 19:28:48,137 - INFO - root - Fetched report for win22
2025-07-31 19:28:48,580 - INFO - root - Fetched report for macos-14
2025-07-31 19:28:49,047 - INFO - root - Fetched report for macos-13
2025-07-31 19:28:49,481 - INFO - root - Fetched report for macos-13-arm64
2025-07-31 19:28:49,927 - INFO - root - Fetched report for macos-15
2025-07-31 19:28:50,371 - INFO - root - Fetched report for macos-15-arm64
